In [ ]:
!pip install Korpora

In [ ]:
from transformers import AutoTokenizer
from Korpora import Korpora



In [ ]:
# Korpora 라이브러리를 사용하여 "kowikitext" 데이터셋을 로드
corpus = Korpora.load("kowikitext")
# 모든 텍스트 데이터를 리스트에 저장합니다. 비어 있는 텍스트는 제외
final_corpus = [i for i in corpus.get_all_texts() if i]



    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/kowikitext
    References :

    한국어 위키피디아의 덤프 데이터를 바탕을 제작한 wikitext 형식의 텍스트 파일입니다.
    학습 및 평가를 위하여 위키페이지 별로 train (99%), dev (0.5%), test (0.5%) 로 나뉘어져있습니다.


    # License
    CC-BY-SA 3.0 which kowiki dump dataset is licensed



[kowikitext] download kowikitext_20200920.train.zip: 494MB [00:06, 70.8MB/s]                          


unzip /root/Korpora/kowikitext/kowikitext_20200920.train


[kowikitext] download kowikitext_20200920.test.zip: 2.51MB [00:00, 7.24MB/s]                            


unzip /root/Korpora/kowikitext/kowikitext_20200920.test


[kowikitext] download kowikitext_20200920.dev.zip: 2.37MB [00:00, 6.51MB/s]                            


unzip /root/Korpora/kowikitext/kowikitext_20200920.dev


KeyboardInterrupt: Interrupted by user

In [ ]:
print(final_corpus[0:10])

In [ ]:
#  리스트를 10,000개씩 나누어 배치를 생성: 학습시 데이터셋을 순차적으로 제공하는 역할
def batch_generation(final_corpus):
    for i in range(0, len(final_corpus), 10000):
        yield final_corpus[i : i + 10000]


In [ ]:
# Tokenizer
tokenizer_dir = 'mistralai/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)



In [ ]:
# 한국어 데이터셋으로 새로운 tokenizer 학습하는 코드
# 토크나이저를 주어진 데이터셋으로 완전히 새로 학습하는 방식
new_tokenizer = tokenizer.train_new_from_iterator(batch_generation(final_corpus), 35000)


In [ ]:
# 한국어 데이터셋으로 학습된 BPE 토크나이저 저장
new_tokenizer.save_pretrained('./new_tokenizer')

In [ ]:
with open("./old_tokenizer/tokenizer.json", "r") as json_file:
    l_info = json.load(json_file)

In [ ]:
with open("./new_tokenizer/tokenizer.json", "r") as json_file:
    n_info = json.load(json_file)

In [ ]:
origin_vocabs_set = set(l_info['model']['vocab'])
expanded_vocab_dict = dict(l_info['model']['vocab'])
expanded_merge_list = list(l_info['model']['merges'])
expanded_vocab_pos = len(expanded_vocab_dict)
merge_rules = n_info['model']['merges']

for token in tqdm(list(filter(lambda x: not x.endswith('\n') and re.match('^[ㄱ-ㅎ가-힣▁]+$', x), n_info['model']['vocab'].keys()))):
    if token not in origin_vocabs_set and len(token) > 1:
        expanded_vocab_dict[token] = expanded_vocab_pos
        expanded_vocab_pos += 1
        expanded_merge_list.extend(list(filter(lambda x: x.replace(' ', '')==token, merge_rules)))

for token in tqdm(list(filter(lambda x: not x.endswith('\n') and re.match('^[ㄱ-ㅎ가-힣▁]+$', x), n_info['model']['vocab'].keys()))):
    if token not in origin_vocabs_set and len(token) == 1:
        expanded_vocab_dict[token] = expanded_vocab_pos
        expanded_vocab_pos += 1

In [ ]:
l=l_info['model']['vocab']

In [ ]:
print(len(expanded_vocab_dict))

In [ ]:
print(len(l)) # 32000
print(len(expanded_vocab_dict)) # 47383

# 기존 토크나이저의 정보를 복사
e_info = copy.deepcopy(l_info)


In [ ]:

# vocab과 merges만 교
e_info['model']['vocab'] = expanded_vocab_dict
e_info['model']['merges'] = expanded_merge_list



In [ ]:
# Write the modified data back to the file
with open("./old_tokenizer/tokenizer.json", "w") as file:
    json.dump(e_info, file, indent=2, ensure_ascii = False)  # Using indent for pretty-printing

e = AutoTokenizer.from_pretrained('./old_tokenizer/')

print(len(e)) # 47383

In [ ]:
e = AutoTokenizer.from_pretrained('./old_tokenizer/')


In [ ]:
e